<a href="https://colab.research.google.com/github/SiYeolJung/BERT-DetectFeel/blob/main/DetectFeel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sklearn
!pip install transformers 
!pip install --user tensorflow-text
!pip install -q tfds-nightly
!pip install matplotlib

     |████████████████████████████████| 2.6 MB 4.2 MB/s 
     |████████████████████████████████| 636 kB 30.5 MB/s 
     |████████████████████████████████| 3.3 MB 26.9 MB/s 
     |████████████████████████████████| 895 kB 34.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 4.0 MB 4.3 MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import os
import re
import shutil
import numpy as np
import pandas as pd
import csv 
import json
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub 
# import tensorflow_text as text 
import tensorflow_datasets as tfds

# from official.nlp import optimization
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import TFBertModel

In [5]:
# 시각화

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [6]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

BATCH_SIZE = 32
NUM_EPOCHS = 20
VALID_SPLIT = 0.2
MAX_LEN = 39 # EDA에서 추출된 Max Length
DATA_IN_PATH = '/content/gdrive/My Drive/Colab Notebooks/'
DATA_OUT_PATH = '/content/gdrive/My Drive/Colab Notebooks/'

In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [8]:
DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH,"FeelTraining.csv")
DATA_TEST_PATH = os.path.join(DATA_OUT_PATH,"FeelTest.csv")
train_data = pd.read_csv(DATA_TRAIN_PATH)##, header = 0, delimiter = '\t', quoting = 2)
train_data = train_data.loc[:,['감정_대분류','사람문장1']]
train_data = train_data.dropna()
# train_data.info()
train_data['감정_대분류'].replace("기쁨 ","기쁨",inplace=True)
train_data['감정_대분류'].replace("불안 ","불안",inplace=True)
train_data['감정_대분류'] = train_data['감정_대분류'].astype('category')
# train_data.info()
train_data['감정_대분류'] = pd.Categorical(train_data['감정_대분류'])
train_data['감정_대분류'] = train_data.감정_대분류.cat.codes

# train_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict = tokenizer.encode_plus(
        text = sent,
        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
        max_length = MAX_LEN,           # Pad & truncate all sentences.
        pad_to_max_length = True,
        return_attention_mask = True   # Construct attn. masks.
        
    )
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id

In [10]:
# train_data = train_data[:1000] # for test

input_ids = []
attention_masks = []
token_type_ids = []
train_data_labels = []

for train_sent, train_label in tqdm(zip(train_data["사람문장1"], train_data["감정_대분류"]), total=len(train_data)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(train_sent, MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        train_data_labels.append(train_label)

    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_feel_input_ids = np.array(input_ids, dtype=int)
train_feel_attention_masks = np.array(attention_masks, dtype=int)
train_feel_type_ids = np.array(token_type_ids, dtype=int)
train_feel_inputs = (train_feel_input_ids, train_feel_attention_masks, train_feel_type_ids)

train_data_labels = np.asarray(train_data_labels, dtype=np.int32) #레이블 토크나이징 리스트

print("# sents: {}, # labels: {}".format(len(train_feel_input_ids), len(train_data_labels)))

  0%|          | 0/40879 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 40879/40879 [00:48<00:00, 834.51it/s]


# sents: 40879, # labels: 40879


In [11]:
# input_id = train_feel_input_ids[0]
# attention_mask = train_feel_attention_masks[0]
# token_type_id = train_feel_type_ids[0]

# print(input_id)
# print(attention_mask)
# print(token_type_id)
# print(tokenizer.decode(input_id))
# print(train_data_labels[0])

In [12]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=6)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [13]:
# 학습 준비하기
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model_name = "DetectFeel"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=5)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

# 학습과 eval 시작
history = cls_model.fit(train_feel_inputs, train_data_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

#steps_for_epoch

print(history.history)



/content/gdrive/My Drive/Colab Notebooks/DetectFeel -- Folder create complete 

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement

In [14]:
test_data = pd.read_csv(DATA_TEST_PATH)##, header = 0, delimiter = '\t', quoting = 2)
test_data = test_data.loc[:,['감정_대분류','사람문장1']]
test_data = test_data.dropna()
# train_data.info()
test_data['감정_대분류'].replace("기쁨 ","기쁨",inplace=True)
test_data['감정_대분류'].replace("불안 ","불안",inplace=True)
print(test_data['감정_대분류'].unique())
test_data['감정_대분류'] = pd.Categorical(test_data['감정_대분류'])
test_data['감정_대분류'] = test_data.감정_대분류.cat.codes
print(test_data['감정_대분류'].unique())
test_data.head()
feeling_key_value = {0:'기쁨',1:'당황',2:'분노',3:'불안',4:'상처',5:'슬픔'}

['분노' '슬픔' '불안' '당황' '상처' '기쁨']
[2 5 3 1 4 0]


In [ ]:
input_ids = []
attention_masks = []
token_type_ids = []
test_data_labels = []

for test_sent, test_label in tqdm(zip(test_data["사람문장1"], test_data["감정_대분류"]), total=len(test_data)):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(test_sent, MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        test_data_labels.append(test_label)

    except Exception as e:
        print(e)
        print(test_sent)
        pass

test_feel_input_ids = np.array(input_ids, dtype=int)
test_feel_attention_masks = np.array(attention_masks, dtype=int)
test_feel_type_ids = np.array(token_type_ids, dtype=int)
test_feel_inputs = (test_feel_input_ids, test_feel_attention_masks, test_feel_type_ids)

test_data_labels = np.asarray(test_data_labels, dtype=np.int32) #레이블 토크나이징 리스트

print("# sents: {}, # labels: {}".format(len(test_feel_input_ids), len(test_data_labels)))

  0%|          | 0/5130 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 5130/5130 [00:01<00:00, 2621.07it/s]

# sents: 5130, # labels: 5130


In [ ]:
results = cls_model.evaluate(test_feel_inputs, test_data_labels, batch_size=1024)
print("test loss, test acc: ", results)

6/6 [==============================] - 23s 4s/step - loss: 1.7514 - accuracy: 0.6123
test loss, test acc:  [1.7514352798461914, 0.6122807264328003]


In [1]:
input_id, attention_mask, token_type_id = bert_tokenizer("사랑해", MAX_LEN)
predict_feel = [np.array([input_id]),np.array([attention_mask]),np.array([token_type_id])]
result = cls_model.predict(predict_feel)
result = np.argmax(result)
result = result.astype(np.int64)
# print(type(result))
print(feeling_key_value[result])

NameError: ignored